Setup

In [12]:
pip install pandas openpyxl pillow numpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import openpyxl
from PIL import Image
from PIL import Image, ImageDraw, ImageChops, ImageFont
import os

In [14]:
sheet = "2025"

df = pd.read_excel("data/Proto-2_Frame_Auto.xlsx", sheet_name=sheet)
df


,full_name,short_name,k-id,birthday,anniversary,department,advance,pro,expert,habit_1,...,habit_4,habit_5,habit_6,shabit_5,th_2,habit_7,habit_8,habit_9,shabit_6,th_3
0,Mathew Sullivan,Mat Sullivan,m.sullivan,May,January,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carter Guerrero,Cart Guerrero,c.guerrero,May,January,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marley Dale,Marl Dale,m.dale,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Harold Griffith,Harry Griffith,h.griffith,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alysha Davidson,Aly Davidson,a.davidson,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Gerald Quinn,Gerald Quinn,g.quinn,May,March,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Aamir Hernandez,Mir Hernandez,m.hernandez,May,March,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mehmet Melendez,Mett Melendez,m.melendez,May,March,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Saoirse Suarez,Sors Suarez,s.suarez,May,March,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Bilal Wilcox,Bil Wilcox,b.wilcox,May,March,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Values Birthdays, Anniversaries, etc.

In [15]:
# change value of this variable
month = "May"
bday = df[(df["birthday"] == month)]
bday_fields = bday[["short_name", "k-id"]]

anniv = df[(df["anniversary"] == month)]
anniv_fields = anniv[["short_name", "k-id"]]


In [16]:
bday_test = bday_fields.to_dict(orient='split', index=False)
bday_data = bday_test['data']

anniv_test = anniv_fields.to_dict(orient='split', index=False)
anniv_data = anniv_test['data']

### Creating groups (to modify)

In [17]:
bday_groups = [] 
while bday_data:
    n = len(bday_data)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            bday_groups.append(bday_data[:size])
            bday_data = bday_data[size:]
            break


print("BIRTHDAY")
print("="*70)
for i, group in enumerate(bday_groups, 1):
    print(f"Group {i} ({len(group)} members):", group)




anniv_groups = [] 
while anniv_data:
    n = len(anniv_data)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            anniv_groups.append(anniv_data[:size])
            anniv_data = anniv_data[size:]
            break

print()
print("Anniversary")
print("="*70)
for i, group in enumerate(anniv_groups, 1):
    print(f"Group {i} ({len(group)} members):", group)



BIRTHDAY
Group 1 (8 members): [['Mat Sullivan', 'm.sullivan'], ['Cart Guerrero', 'c.guerrero'], ['Marl Dale', 'm.dale'], ['Harry Griffith', 'h.griffith'], ['Aly Davidson', 'a.davidson'], ['Gerald Quinn', 'g.quinn'], ['Mir Hernandez', 'm.hernandez'], ['Mett Melendez', 'm.melendez']]
Group 2 (5 members): [['Sors Suarez', 's.suarez'], ['Bil Wilcox', 'b.wilcox'], ['Lu Jefferson', 'l.jefferson'], ['Emman Hardy', 'e.hardy'], ['Jac Roberts', 'j.roberts']]

Anniversary
Group 1 (2 members): [['Eby Ho', 'e.ho'], ['Dev Dunn', 'd.dunn']]


### Image Coordinates

In [18]:
single_y = 418
top_y = 328
bottom_y = 732

coord_arr_8 = [(51, top_y),(500, top_y),(948, top_y),(1396, top_y),
               (266, bottom_y),(714, bottom_y),(1163, bottom_y),(1611, bottom_y)]
coord_arr_7 = [(91, top_y),(589, top_y),(1087, top_y),(1585, top_y),
               (308, bottom_y),(838, bottom_y),(1367, bottom_y)]
coord_arr_6 = [(203, top_y),(733, top_y),(1262, top_y),
               (454, bottom_y),(983, bottom_y),(1513, bottom_y)]
coord_arr_5 = [(203, top_y), (838, top_y), (1472, top_y), 
               (518, bottom_y), (1158, bottom_y)]
coord_arr_4 = [(172, single_y),(616, single_y),(1060, single_y),(1504, single_y)]
coord_arr_3 = [(268, single_y),(838, single_y),(1408, single_y)]
coord_arr_2 = [(565, single_y),(1111, single_y)]
coord_arr_1 = [(838, single_y)]

## Notes

TODO: 
- calculate image and text offset to make the anchor point be the middle
- read the name and image path from the excel file 
- use the appropriate canvas when of number of people 

TODO: 
- Make this into a video, save it and get the duration 
- Set this duration in the software upload and observe how it will play

TODO:
- create folder for image and file in the downloads folder for easy access 
- detect if the file is an image or excel file 
- move them in their respective folders inside the project 

# Main Process

In [19]:
from PIL import Image, ImageDraw, ImageFont
import os

# Values to change
outer_fill_color = "#ed6e38"
inner_fill_color = "#ffcf00"
font = ImageFont.truetype("DejaVuSansMono.ttf", size=25) # note: windows font starts with small letter
department = "IT"

# These two are hand in hand, make sure to change them at the same time
# Comment in and out the category fields (for improvement)
# Bday
# category = "bday" 
# data_group = bday_groups

# Anniversary
category = "anniv" 
data_group = anniv_groups


# Coordinate mapping
coord_map = {
    8: coord_arr_8,
    7: coord_arr_7,
    6: coord_arr_6,
    5: coord_arr_5,
    4: coord_arr_4,
    3: coord_arr_3,
    2: coord_arr_2,
    1: coord_arr_1,
}

IMAGE_SIZE = 245

# Make an image circular with transparency
def make_circular(image_path):
    img = Image.open(image_path).convert("RGBA")
    width, height = img.size

    mask = Image.new("L", (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0, width, height), fill=255)
    img.putalpha(mask)
    return img

def create_border(diamter, x0, y0):
    x1, y1 = x0 + diamter, y0 + diamter
    bounding_box = x0, y0, x1, y1 
    return bounding_box

def calculate_border(IMAGE_SIZE, image_x, image_y, size_difference, position_offset):
    diameter = IMAGE_SIZE + size_difference
    x_coord_diff = y_coord_diff = position_offset
    x_offset = image_x - x_coord_diff
    y_offset = image_y - y_coord_diff
    return diameter, x_offset, y_offset


# Create the final composite image
def create_image(names, coords, usernames, department, category, counter, background_path):
    canvas = Image.open(background_path).convert("RGBA")
    draw = ImageDraw.Draw(canvas)

    for (image_x, image_y), username, name in zip(coords, usernames, names):
        image_path = f"images/{username}.jpg"
        try:
            circular_image = make_circular(image_path)
        except FileNotFoundError:
            circular_image = make_circular("images/person-holder.jpg")

        # Setting values for inner and outer border size and coordinate
        INNER_BORDER_SIZE_OFFSET = 29
        INNER_BORDER_POSITION_OFFSET = 15
        OUTER_BORDER_SIZE_OFFSET = 63
        OUTER_BORDER_POSITION_OFFSET = 32
        
        inner_diameter, inner_x_offset, inner_y_offest = calculate_border(IMAGE_SIZE, image_x, image_y, 
                                                                          INNER_BORDER_SIZE_OFFSET, INNER_BORDER_POSITION_OFFSET)
        outer_diameter, outer_x_offset, outer_y_offest = calculate_border(IMAGE_SIZE, image_x, image_y,
                                                                          OUTER_BORDER_SIZE_OFFSET, OUTER_BORDER_POSITION_OFFSET)

        #  Applying the values and drawing it to the canvas
        outer_bounding_box = create_border(outer_diameter, outer_x_offset, outer_y_offest)
        draw.ellipse(outer_bounding_box, fill=outer_fill_color)

        inner_bounding_box = create_border(inner_diameter, inner_x_offset, inner_y_offest)
        draw.ellipse(inner_bounding_box, fill=inner_fill_color)

        resized_image = circular_image.resize((245, 245), Image.LANCZOS)
        canvas.paste(resized_image, (image_x, image_y), resized_image)

        # Configuring text values 
        TEXT_X_OFFSET = 120
        TEXT_Y_OFFSET = 300
        text_position = (image_x + TEXT_X_OFFSET, image_y + TEXT_Y_OFFSET)
        draw.text(text_position, name, fill=(0, 0, 0), font=font, anchor="mm")

    # Saving File
    output_name = f"[{department}]-{category}-{counter}.jpeg"
    os.makedirs("output", exist_ok=True)
    canvas.convert("RGB").save(os.path.join(f"output", output_name), "JPEG")


# Runner loop
counter = 0
for group in data_group:
    group_size = len(group)    
    coords = coord_map.get(group_size)
    background_path = f"backgrounds/{category}.png"
    if coords and background_path:

        
        usernames = [person[1].replace(".", "-") for person in group]
        names = [person[0] for person in group]
        create_image(names, coords, usernames, department, category, counter, background_path)
        counter += 1
    else:
        print(f"No coordinate layout or background found for group size {group_size}")



TODO:
- This only caters to the one category which is bday.
- Create the other category as well

In [20]:
pip install moviepy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install --upgrade --force-reinstall moviepy


  Using cached moviepy-2.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached proglog-0.1.12-py3-none-any.whl.metadata (794 bytes)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached moviepy-2.1.2-py3-none-any.whl (126 kB)
Using cached decorator-5.2.1-py3-none-any.whl (9.2 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl (29.5 MB)
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 M

In [22]:
from moviepy.editor import ImageClip, concatenate_videoclips
import os

# Set your image folder and output file name
image_folder = "output"  # Replace with your folder path
output_video = "slideshow.mp4"
fps = 24
duration_per_image = 3  # seconds

# Get list of image files
image_files = sorted([
    os.path.join(image_folder, file)
    for file in os.listdir(image_folder)
    if file.lower().endswith(('.png', '.jpg', '.jpeg'))
])

# Create a list of ImageClips
clips = [
    ImageClip(img).set_duration(duration_per_image)
    for img in image_files
]

# Concatenate all the image clips into a final video
slideshow = concatenate_videoclips(clips, method="compose")
slideshow.write_videofile(output_video, fps=fps)


ModuleNotFoundError: No module named 'moviepy.editor'

In [ ]:
pip show moviepy

In [23]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 1.8 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import cv2
import os

image_folder = "output"  # Folder with your images
output_video = "slideshow.mp4"
fps = 24                 # Frames per second
duration = 3             # Duration per image in seconds

image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))])

# Read the first image to get dimensions
first_image = cv2.imread(os.path.join(image_folder, image_files[0]))
height, width, layers = first_image.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    frame = cv2.imread(image_path)
    
    # Write the same frame multiple times to create duration
    for _ in range(int(fps * duration)):
        video.write(frame)

video.release()
print("Video saved as", output_video)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video saved as slideshow.mp4


In [ ]:
import cv2
import os

image_folder = "output"
output_video = "custom_slideshow.avi"
fps = 24  # frames per second

# List of (filename, duration_in_seconds)
image_durations = [
    ("image1.jpg", 3),
    ("image2.jpg", 5),
    ("image3.jpg", 2),
]

# Read the first image to get dimensions
first_image = cv2.imread(os.path.join(image_folder, image_durations[0][0]))
height, width, layers = first_image.shape

# Create the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Write frames
for image_name, duration in image_durations:
    image_path = os.path.join(image_folder, image_name)
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Skipping missing image: {image_name}")
        continue

    frame_count = int(fps * duration)

    for _ in range(frame_count):
        video.write(frame)

video.release()
print("Custom-duration slideshow saved as:", output_video)


In [45]:
import os

def print_folder_items(folder_path):
    """Prints all items in a folder."""

    file_path = "image_list.txt"
    with open(file_path, "w") as file:
        pass # The file is opened in write mode and immediately closed, clearing its content.
    
    try:
        items = os.listdir(folder_path)
        for item in items:
          
          file_path = "image_list.txt"
          string_to_append = f"{item}\n"

          with open(file_path, "a") as file:
            file.write(string_to_append)

    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
    except NotADirectoryError:
      print(f"Error: '{folder_path}' is not a directory.")


folder_path = "output"
file_list = print_folder_items(folder_path)


#open and read the file after the appending:
with open("image_list.txt") as f:
  print(f.read())

[IT]-anniv-0.jpeg
[IT]-bday-1.jpeg
[IT]-bday-0.jpeg



User Manual:
- Update the excel from time to time
    - add new hires and their information
- Run the program

Video creation
- Copy the files, rearrange them and set the duration for each. 
- paste into the video creation script